In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import matplotlib
import os
from IPython.display import HTML
import numpy as np

# piezo list
piezo = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'RAmont', 'RAval']

df_piezo_dict = {}

# read the excel file and add to the dictionary
for p in piezo:
    df_piezo_dict[p] = pd.read_excel('piezometry.xlsx', sheet_name=p)

# read piezo coordinates csv
csv_dir = os.path.join('data', 'bip', 'Piezometres')
piezo_coords = pd.read_csv(os.path.join(csv_dir, 'piezo_coordinates.csv'))

# read profile coordinates csv
profile_coords = pd.read_csv(os.path.join(csv_dir, 'profile_coordinates.csv'))

In [ ]:
for k, v in df_piezo_dict.items():
    print(k)
    # order by date
    v = v.sort_values(by='date_time')
    # print duplicates
    print(v[v.duplicated(subset='date_time', keep=False)])

In [ ]:
# combine for each piezo in piezo_dict to keep only level_ngf_m as level and date_time. The column 'name' will be used to identify the piezometer
df = pd.DataFrame()
for k, v in df_piezo_dict.items():
    v['name'] = k
    df = pd.concat([df, v], axis=0, ignore_index=True)
df = df[['date_time', 'level_ngf_m', 'name']]
df = df.sort_values(by='date_time', ascending=True)
df = df.rename(columns={'date_time':'dt', 'level_ngf_m': 'level'})
# add the coordinates of the piezometer with cols Est = x and Nord = y by merging with piezo_coords and unsing Point = name
df = pd.merge(df, piezo_coords, left_on='name', right_on='Point', how='left')
df = df.rename(columns={'Est': 'x', 'Nord': 'y', 'Alti': 'z'})
df = df[['dt', 'level', 'name', 'x', 'y', 'z']]
df

In [ ]:
# df z elevation piezometer
df_z = df.copy()
# group by name and get the first value of x, y and z
df_z = df_z.groupby('name').first().reset_index()
df_z = df_z[['name', 'x', 'y', 'z']]

In [ ]:
# prepare the dataframe for animation

# filter df by date from 2024-03-02 to 2024-04-17
df_event = df[(df['dt'] >= '2024-03-02') & (df['dt'] <= '2024-04-17')]

# create dataframe but mean the level for each day and each piezometer
df_day = df_event.copy()
df_day['dt'] = df_day['dt'].dt.date
df_day = df_day.groupby(['dt', 'name', 'x', 'y', 'z']).mean().reset_index()

# create dateframe with distance between piezometer A-C, D-G, H-K, L-N
df_dist = pd.DataFrame({'name1': ['A', 'D', 'H', 'L'], 'name2': ['C', 'G', 'K', 'N'], 'dist': [0, 0, 0, 0]})
df_dist = pd.merge(df_dist, df_z, left_on='name1', right_on='name', how='left')
df_dist = pd.merge(df_dist, df_z, left_on='name2', right_on='name', how='left')
df_dist['dist'] = np.sqrt((df_dist['x_x'] - df_dist['x_y'])**2 + (df_dist['y_x'] - df_dist['y_y'])**2)
df_dist = df_dist[['name1', 'name2', 'dist']]
df_dist

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# view angle
ax.view_init(10, -90)

# Function to update the plot
def update(frame):
    ax.cla()  # Clear the axis

    # add profile coordinates
    ax.scatter(profile_coords['x'], profile_coords['y'], profile_coords['z'], color='grey')

    # add piezo z coordinates scatter with df_day x, y and z
    ax.scatter(df_z['x'], df_z['y'], df_z['z'], color='grey')
    
    # Get data for the current frame
    current_time = df_day['dt'].unique()[frame]
    data_current_time = df_day[df_day['dt'] == current_time]

    # Extract coordinates
    x = data_current_time['x'].values
    y = data_current_time['y'].values
    level = data_current_time['level'].values
    
    # Plot points
    ax.scatter(x, y, level, color = 'blue', s=80, marker='o')
    
    # # Plot lines
    # ax.plot([x[0], x[1]], [y[0], y[1]], [level[0], level[1]], color='blue')
    # ax.plot([x[1], x[2]], [y[1], y[2]], [level[1], level[2]], color='blue')
    
    # Set limits
    ax.set_xlim(df_day['x'].min()-1, df_day['x'].max()+1)
    ax.set_ylim(df_day['y'].min()-1, df_day['y'].max()+1)
    ax.set_zlim(df_day['level'].min()-1, df_day['level'].max()+1)
    
    # Add a cursor for the date
    ax.text2D(0.05, 0.95, str(current_time), transform=ax.transAxes)

    # add label by name
    for i, txt in enumerate(data_current_time['name']):
        ax.text(x[i], y[i], level[i], txt, color='red')

# Create the animation
ani = FuncAnimation(fig, update, frames=len(df_day['dt'].unique()), interval=1000)

# Display the plot
HTML(ani.to_jshtml())

# save the animation
# ani.save('animation.gif', writer='PillowWriter', fps=2)
